In this notebook, we will face how to measure the state of a quantum system.
And we'll understand the role of observables in quantum mechanics.

In [1]:
import pennylane as qml
from pennylane import numpy as np


We provide a simple example of two qbits, where the first one is in the state |0>, and the second one is in the state |1>.
In order to set this configuration, we leave inalterated the first qbit, and we apply a PauliX gate to the second one.

In [2]:
dev1 = qml.device("default.qubit", wires=2, shots=1)
@qml.qnode(dev1)

def simpleCircuit():
    qml.PauliX(wires=1)

    return qml.sample(), qml.counts(), qml.probs()

stateArray, stateDict, stateArrayprobs = simpleCircuit()
print(stateArray)
print(stateDict)
print(stateArrayprobs)

[0 1]
{'01': tensor(1, requires_grad=True)}
[0. 1. 0. 0.]


You want to take a single sample from both qubits: Exe1
Both of 2 qubits are in state 0, when we implement the hadamrd gate on the first qubit, it becomes a superposition of 0 and 1 with the same ampitude.
If when i do the measurement it is on |0> state then nothing hapend to the second qubit.
If, in the moment i compute the measurement, the first qubit is in tate |1> we flip the second qubit from 0 to 1


In [3]:
dev1 = qml.device("default.qubit" , wires=2, shots=100)

@qml.qnode(dev1)
def circuit():
    """
    This quantum function implements the circuit shown above
    and should return a sample from all qubits
    """

    qml.Hadamard(wires=0)
    qml.CNOT(wires=[0,1])

    return qml.sample() , qml.counts()

arrayCircuit, dictionaryCircuit = circuit()
print(dictionaryCircuit)


{'00': tensor(46, requires_grad=True), '11': tensor(54, requires_grad=True)}


Probs relative to full circuit, first wire & second wire

In [ ]:
dev2 = qml.device("default.qubit", wires=2)
@qml.qnode(dev2)
def circuit2():
    qml.PauliX(wires=0)
    return qml.probs() , qml.probs(wires=[0]), qml.probs(wires=[1])

probFull, prob0, prob1  =  circuit2()
print(probFull, "Probability for: |00> |01> |10> |11>\n")
print(prob0, "Probability of first qubit in wire 0: |0> |1>\n")
print(prob1, "Probability of second qubit in wire 1: |0> |1>\n")

[0. 0. 1. 0.] 

[0. 1.] 

[1. 0.] 



EXPECTATION VALUES are the most used measurements especially in optimization and quantum machine learning, because they provide informations about the state of the system.

REMEMBER: the expectation value for an operator OP on a particular state |ψ⟩ is defined as: ⟨ψ|OP|ψ⟩
If we take back the circuit from previous code, we have two qubits, initially both are in the state |0>. Then we apply a x gate on the second so it flips to |1> state. 

If we measure the expected value of Z operator on the first qubit, we expect to get 1.

In [8]:
dev3 = qml.device("default.qubit", wires=2, shots = 1)
@qml.qnode(dev3)

def circuit3():
    qml.PauliX(wires = 1)
    return qml.expval(qml.PauliZ(0))

expectedZvalue = circuit3()
print(expectedZvalue)

1.0


In [10]:
dev4 = qml.device("default.qubit", wires = 2)

@qml.qnode(dev4)
def expval_circuit():
    
    qml.RY(np.pi/4, wires = 0)
    qml.RX(np.pi/3, wires = 1)
    qml.CNOT(wires = [0, 1])
    
    return qml.expval(1/3*qml.PauliZ(0) @ qml.PauliZ(1)) #---> tensor product between the two Pauli Z operators computed on the 2 different qubits


expectedValue= expval_circuit()
print(expectedValue)

0.16666666666666669


We can implement also a custom matrix, and compute the expected value of the custom matrix on a specific qubit.

In [12]:

dev5 = qml.device("default.qubit", wires = 2)

A = np.array([[1, 0], [0, -1]])

@qml.qnode(dev5)
def circuit5():
    """
    This quantum function implements a Bell state and
    should return the expectation value the observable
    corresponding to the matrix A applied to the first qubit
    """
    qml.Hadamard(wires=0)
    qml.CNOT(wires=[0,1])
    return qml.expval(qml.Hermitian(A, wires=0))  #---> <0|A|0>

expectedAvalue = circuit5()
print(expectedAvalue)


0.0


We can implement expval in parallel for all the qubits

In [22]:
dev6 = qml.device("default.qubit", wires=2,)
@qml.qnode(dev6)

def circuit():
    qml.Hadamard(wires=0)
    qml.CNOT(wires=[0,1])
    return qml.expval(qml.PauliZ(0)) , qml.expval(qml.PauliZ(1)) #It provides a tuple

bothComputed, bothcomputed2 = circuit()
print(bothComputed)
print(bothcomputed2)


0.0
0.0


In [26]:
dev7 = qml.device("default.qubit", wires=2)
@qml.qnode(dev7)

def circuit():
    qml.Hadamard(wires=0)
    qml.CNOT(wires=[0,1])
    return qml.probs()

a = circuit()
print(a)

[0.5 0.  0.  0.5]
